# Basic RNA circuit simulation 



In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from bioreaction.simulation.basic_sim import convert_model

import jax
import numpy as np
import pandas as pd
import os
import sys

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-paper')
os.environ["TF_CPP_MIN_LOG_LOVEL"] = "0"
jax.config.update('jax_platform_name', 'gpu')


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)


from src.utils.data.data_format_tools.common import load_json_as_dict
from src.utils.common.setup_new import prepare_config, construct_circuit_from_cfg
from src.utils.circuit.agnostic_circuits.circuit_manager_new import CircuitModeller
from src.utils.evolution.evolver import load_mutations
from tests_local.shared import five_circuits

config = load_json_as_dict('../tests_local/configs/simple_circuit.json')


In [7]:



def update_model_rates(model, a=None, d=None, ka=None, kd=None):
    for i, r in enumerate(model.reactions):
        if not r.input:  # 0 -> RNA
            if a is not None:
                model.reactions[i].forward_rate = a[model.species.index(
                    r.output[0])]
                model.reactions[i].reverse_rate = 0
        elif not r.output:  # RNA -> 0
            if d is not None:
                model.reactions[i].forward_rate = d[model.species.index(
                    r.input[0])]
                model.reactions[i].reverse_rate = 0
        else: # unbound RNA -> bound RNA
            if ka is not None:
                model.reactions[i].forward_rate = ka[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
            if kd is not None:
                model.reactions[i].reverse_rate = kd[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
    return model


def make_params(model, scale_rates=True):
    sim_model = convert_model(model)

    if scale_rates:
        m = np.max([sim_model.forward_rates.max(), sim_model.reverse_rates.max()])
    else:
        m = 1
    inputs = sim_model.inputs
    outputs = sim_model.outputs
    forward_rates = sim_model.forward_rates/m
    reverse_rates = sim_model.reverse_rates/m
    return inputs, outputs, forward_rates, reverse_rates, m

Use the 5 tester circuits :)

In [8]:
config = prepare_config(config)
config['include_prod_deg'] = True
config['simulation']['use_rate_scaling'] = False

q = 2
p = 30
config['molecular_params_factor'] = p
config['simulation']['interaction_factor'] = 1
config['molecular_params']['creation_rate'] = config['molecular_params']['creation_rate'] * q
config['molecular_params']['creation_rate' + '_per_molecule'] = config['molecular_params']['creation_rate' + '_per_molecule'] * q
# config['simulation']['dt'] = 0.1
# config['molecular_params']['creation_rate'] = config['molecular_params']['creation_rate'] * p
# config['molecular_params']['degradation_rate'] = config['molecular_params']['degradation_rate'] * p
circuits, config, result_writer = five_circuits(config)

experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/binding_rates_dissociation/0_weak_binding_rates_dissociation.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/eqconstants/0_weak_eqconstants.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/energies/0_weak_energies.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/binding_sites/0_weak_binding_sites.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/binding_rates_dissociation/1_med_weak_binding_rates_dissociation.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead

In [9]:


def load_circuit(top_dir, circuit_name, config):
    dp = os.path.join(top_dir, 'circuits', circuit_name + '.fasta')
    interactions = {'binding_rates_association': config['molecular_params']['association_binding_rate' + '_per_molecule'],
                    'binding_rates_dissociation': os.path.join(top_dir, 'binding_rates_dissociation', circuit_name + '_' + 'binding_rates_dissociation' + '.csv'),
                    'eqconstants': os.path.join(top_dir, 'eqconstants', circuit_name + '_' + 'eqconstants' + '.csv'),
                    'energies': os.path.join(top_dir, 'energies', circuit_name + '_' + 'energies' + '.csv'),
                    'binding_sites': os.path.join(top_dir, 'binding_sites', circuit_name + '_' + 'binding_sites' + '.csv')}

    return construct_circuit_from_cfg({
        'data_path': dp,
        'interactions': interactions
    }, config)

# tester_circuit = load_circuit(top_dir='data/ensemble_mutation_effect_analysis/2023_04_04_092945/generate_species_templates',
#              circuit_name='toy_mRNA_circuit_1844', config=config)


tester_circuits = []
for name in ['toy_mRNA_circuit_21566', 'toy_mRNA_circuit_24706', 'toy_mRNA_circuit_19768', 'toy_mRNA_circuit_2983', 'toy_mRNA_circuit_22658']:
    fn = os.path.join('..', 'data/ensemble_mutation_effect_analysis/2023_04_11_192013/mutation_effect_on_interactions_signal', name, 'mutations.csv')
    tester_circuit = load_circuit(top_dir='data/ensemble_mutation_effect_analysis/2023_04_04_092945/generate_species_templates',
                                        circuit_name=name, config=config)
    tester_circuit = load_mutations(tester_circuit, fn)
    tester_circuits.append(tester_circuit)

# config['interactions'] = {}
# config['interactions'] = None
# tester_circuit = construct_circuit_from_cfg(
#     {'data_path': '../tester.fasta'}, config)
[circuits.append(tester_circuit) for tester_circuit in tester_circuits]


[None, None, None, None, None]

In [10]:
circuits[-1].mutations

{Species: RNA_0: {'RNA_0_m1-0': <src.utils.evolution.mutation.Mutations at 0x7f42ca776a40>,
  'RNA_0_m1-1': <src.utils.evolution.mutation.Mutations at 0x7f42ca776fb0>,
  'RNA_0_m1-2': <src.utils.evolution.mutation.Mutations at 0x7f42ca777250>,
  'RNA_0_m1-3': <src.utils.evolution.mutation.Mutations at 0x7f42ca7770d0>,
  'RNA_0_m1-4': <src.utils.evolution.mutation.Mutations at 0x7f42ca776d10>,
  'RNA_0_m1-5': <src.utils.evolution.mutation.Mutations at 0x7f42ca776dd0>,
  'RNA_0_m1-6': <src.utils.evolution.mutation.Mutations at 0x7f42ca776b90>,
  'RNA_0_m1-7': <src.utils.evolution.mutation.Mutations at 0x7f42ca776c80>,
  'RNA_0_m3-0': <src.utils.evolution.mutation.Mutations at 0x7f42ca776f50>,
  'RNA_0_m3-1': <src.utils.evolution.mutation.Mutations at 0x7f42ca777370>,
  'RNA_0_m3-2': <src.utils.evolution.mutation.Mutations at 0x7f42ca7772b0>,
  'RNA_0_m3-3': <src.utils.evolution.mutation.Mutations at 0x7f42ca776fe0>,
  'RNA_0_m3-4': <src.utils.evolution.mutation.Mutations at 0x7f42ca7770a

In [16]:

circuit_modeller = CircuitModeller(result_writer=result_writer, config=config)
circuits = circuit_modeller.batch_circuits(
    circuits=circuits, methods={"compute_interactions": {}},
    batch_size=config['simulation']['batch_size'])
i = -1
# for i in range(len(circuits)):
#     circuits[i].interactions.binding_rates_association = circuits[i].interactions.binding_rates_association * q
#     circuits[i].interactions.binding_rates_dissociation = circuits[i].interactions.binding_rates_dissociation * q
#     circuits[i].update_species_simulated_rates(circuits[i].interactions)


circuit_manager_new.py:batch_circuits():565: 	From 10 circuits, a total of 610 mutated circuits will be simulated. WARNING
circuit_manager_new.py:batch_circuits():572: 		Starting new round of viable circuits (0 - 10 / 10) WARNING
circuit_manager_new.py:batch_circuits():590: 		Making subcircuits 0 - 610 took 1.214347s WARNING
circuit_manager_new.py:batch_circuits():596: 	Batching 0 - 100 circuits (out of 0 - 610 (total: 610)) (Circuits: 0 - 10 of 10) WARNING
circuit_manager_new.py:run_batch():627: 		Running 100 Subcircuits - 0_weak: compute_interactions WARNING
circuit_manager_new.py:run_batch():663: 		Method compute_interactions took 5.300405s WARNING
circuit_manager_new.py:batch_circuits():596: 	Batching 100 - 200 circuits (out of 0 - 610 (total: 610)) (Circuits: 0 - 10 of 10) WARNING
circuit_manager_new.py:run_batch():627: 		Running 100 Subcircuits - toy_mRNA_circuit_21566: compute_interactions WARNING
circuit_manager_new.py:run_batch():663: 		Method compute_interactions took 5.73721

In [17]:
circuits[i].interactions.binding_rates_dissociation

array([[0.00052282, 0.22631998, 0.01330353],
       [0.22631998, 0.22631998, 0.00028829],
       [0.01330353, 0.00028829, 0.22631998]])

In [18]:
circuits[i].interactions.binding_rates_association

array([[0.00150958, 0.00150958, 0.00150958],
       [0.00150958, 0.00150958, 0.00150958],
       [0.00150958, 0.00150958, 0.00150958]])

In [19]:
config['molecular_params']['creation_rate']

141.0

In [20]:
config['molecular_params']['degradation_rate']


0.3525

In [22]:

circuits = circuit_modeller.batch_circuits(
    circuits=circuits, 
    batch_size=config['simulation']['batch_size'],
    methods={
        # "compute_interactions": {},
        "init_circuits": {'batch': True},
        'simulate_signal_batch': {'ref_circuit': None,
                                  'batch': config['simulation']['use_batch_mutations']},
        'write_results': {'no_visualisations': False, # config['experiment']['no_visualisations'],
                          'no_numerical': False} #config['experiment']['no_numerical']}
    }
)

circuit_manager_new.py:batch_circuits():565: 	From 10 circuits, a total of 610 mutated circuits will be simulated. WARNING
circuit_manager_new.py:batch_circuits():572: 		Starting new round of viable circuits (0 - 10 / 10) WARNING
circuit_manager_new.py:batch_circuits():590: 		Making subcircuits 0 - 610 took 1.381139s WARNING
circuit_manager_new.py:batch_circuits():596: 	Batching 0 - 100 circuits (out of 0 - 610 (total: 610)) (Circuits: 0 - 10 of 10) WARNING
circuit_manager_new.py:run_batch():627: 		Running 100 Subcircuits - 0_weak: init_circuits WARNING


Done:  0:00:02.941173


circuit_manager_new.py:run_batch():663: 		Method init_circuits took 3.152561s WARNING
circuit_manager_new.py:run_batch():627: 		Running 100 Subcircuits - 0_weak: simulate_signal_batch WARNING
circuit_manager_new.py:prepare_internal_funcs():495: Signal differs between circuits, but only first signal used for simulation. WARNING
circuit_manager_new.py:simulate_signal_batch():343: 		Simulating signal took 3.851538s WARNING


Done:  0:00:03.831693


circuit_manager_new.py:simulate_signal_batch():401: 		Calculating analytics took 4.515683s WARNING
circuit_manager_new.py:run_batch():663: 		Method simulate_signal_batch took 8.583727s WARNING
circuit_manager_new.py:run_batch():627: 		Running 100 Subcircuits - 0_weak: write_results WARNING
circuit_manager_new.py:run_batch():663: 		Method write_results took 65.819219s WARNING
circuit_manager_new.py:batch_circuits():596: 	Batching 100 - 200 circuits (out of 0 - 610 (total: 610)) (Circuits: 0 - 10 of 10) WARNING
circuit_manager_new.py:run_batch():627: 		Running 100 Subcircuits - toy_mRNA_circuit_21566: init_circuits WARNING


...

circuit_manager_new.py:simulate_signal_batch():401: 		Calculating analytics took 3.471347s WARNING
circuit_manager_new.py:run_batch():663: 		Method simulate_signal_batch took 9.238276s WARNING
circuit_manager_new.py:run_batch():627: 		Running 100 Subcircuits - toy_mRNA_circuit_22658: write_results WARNING
circuit_manager_new.py:run_batch():663: 		Method write_results took 68.883547s WARNING
circuit_manager_new.py:batch_circuits():596: 	Batching 600 - 700 circuits (out of 0 - 610 (total: 610)) (Circuits: 0 - 10 of 10) WARNING
circuit_manager_new.py:run_batch():627: 		Running 10 Subcircuits - toy_mRNA_circuit_22658: init_circuits WARNING
circuit_manager_new.py:run_batch():663: 		Method init_circuits took 2.200978s WARNING
circuit_manager_new.py:run_batch():627: 		Running 10 Subcircuits - toy_mRNA_circuit_22658: simulate_signal_batch WARNING


Done:  0:00:02.128690


circuit_manager_new.py:simulate_signal_batch():343: 		Simulating signal took 3.035769s WARNING


Done:  0:00:03.014215


circuit_manager_new.py:simulate_signal_batch():401: 		Calculating analytics took 1.3477s WARNING
circuit_manager_new.py:run_batch():663: 		Method simulate_signal_batch took 6.547909s WARNING
circuit_manager_new.py:run_batch():627: 		Running 10 Subcircuits - toy_mRNA_circuit_22658: write_results WARNING
circuit_manager_new.py:run_batch():663: 		Method write_results took 6.227059s WARNING
circuit_manager_new.py:batch_circuits():610: Single batch: 0:07:57.679896 
Projected time: 477.679896s 
Total time: 0:07:57.679907 WARNING


Log

1.
q = 5
p = 2
-> not great

2. 
q = 2
p = 5
-> good, the weak ones could be more step-like

3. 
q = 3
p = 5
-> Same-looking slope for signal, but doesn't reach as high as 2. 

4. 
q = 2
p = 10
-> better, more step-like on strong, but still a bit dragging on weaker ones

5. 
q = 2
p = 15
-> even better, more step-like, but no overshoot like you get in the no prod/deg

6. 
q = 5
p = 50
-> spiky, but now a bit too dominating with prod / deg




